<a href="https://colab.research.google.com/github/ayanoglu/REDGPA/blob/GPR/REDGPAmain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from numpy import array
from sklearn import preprocessing
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
np.set_printoptions(linewidth=160)
import tensorflow as tf
import csv
import math
import matplotlib.pyplot as plt
from decimal import *
from google.colab import files
import plotly.graph_objects as go
import plotly.express as px
import itertools
from itertools import combinations
import time
from sklearn.metrics import mean_squared_error
import io

uploaded = files.upload()

Saving Survey_numerical_score_GPA.csv to Survey_numerical_score_GPA.csv


In [ ]:
DF = pd.read_csv(io.BytesIO(uploaded['Survey_numerical_score_GPA.csv']), header=None, skip_blank_lines=True)

In [ ]:
DF.head()

In [ ]:
DF.describe()

In [ ]:
DF.isna().sum()

In [ ]:
DF = DF.dropna(axis=1)
DF.isna().sum()
X = DF.iloc[:,:-1]
y = DF.iloc[:,-1]
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,6,6,6,6,6,6,7,5,3,3,5,5,6,4,4,6,5,7,6,6,6,3,5,5,4,5,5,5,6,5,5,1,1,1,1,1,3,3,4,1,5,5
1,7,7,6,6,6,7,7,7,5,6,6,2,2,4,7,6,7,7,4,6,5,6,2,2,6,6,6,2,6,4,4,4,7,3,1,4,1,1,4,4,5,2
2,7,7,7,7,7,7,7,7,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,6,6,6,1,4,3,5,5,1,1,1,1,4,4,4,2
3,6,6,6,5,6,6,5,7,3,7,4,4,4,3,3,3,7,5,5,5,4,5,4,5,5,5,5,4,4,1,1,1,7,1,1,2,2,2,2,1,4,5
4,6,7,6,6,6,6,7,5,4,3,5,1,1,5,7,7,6,2,2,7,5,4,1,1,6,7,4,5,1,4,4,4,7,1,1,1,1,1,5,4,4,2


In [ ]:
X_norm = X*4/10
X_norm.shape

(187, 42)

In [ ]:
X_norm.skew() # -0.5 to 0.5 values are moderatly skewed, meaning closer to a normal distribution.

In [ ]:
DF2 = pd.concat((X_norm, y), axis = 1)
DF2.shape

(187, 43)

In [ ]:
#train_Input, val_Input, test_input = np.split(DF2, [int(.6 * len(DF2)), int(.8 * len(DF2))])
train_Input, test_input = np.split(DF2, [int(.8 * len(DF2))])

X_train_Input = train_Input.iloc[:,:-1]
y_train= train_Input.iloc[:,-1]
# X_val_Input = val_Input.iloc[:,:-1]
# y_val= val_Input.iloc[:,-1]
X_test_Input = test_input.iloc[:,:-1]
y_test= test_input.iloc[:,-1]

In [ ]:
print(f'Length of training set is', len(X_train_Input))
#print(f'Length of validation set is', len(X_val_Input))
print(f'Length of test set is', len(X_test_Input))

Length of training set is 149
Length of test set is 38


In [ ]:
X_train_Input.shape

(149, 42)

In [ ]:
X_train_Input.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,2.4,2.4,2.4,2.4,2.4,2.4,2.8,2.0,1.2,1.2,2.0,2.0,2.4,1.6,1.6,2.4,2.0,2.8,2.4,2.4,2.4,1.2,2.0,2.0,1.6,2.0,2.0,2.0,2.4,2.0,2.0,0.4,0.4,0.4,0.4,0.4,1.2,1.2,1.6,0.4,2.0,2.0,NaN
1,2.8,2.8,2.4,2.4,2.4,2.8,2.8,2.8,2.0,2.4,2.4,0.8,0.8,1.6,2.8,2.4,2.8,2.8,1.6,2.4,2.0,2.4,0.8,0.8,2.4,2.4,2.4,0.8,2.4,1.6,1.6,1.6,2.8,1.2,0.4,1.6,0.4,0.4,1.6,1.6,2.0,0.8,NaN
2,2.8,2.8,2.8,2.8,2.8,2.8,2.8,2.8,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.8,2.4,2.4,2.4,0.4,1.6,1.2,2.0,2.0,0.4,0.4,0.4,0.4,1.6,1.6,1.6,0.8,NaN
3,2.4,2.4,2.4,2.0,2.4,2.4,2.0,2.8,1.2,2.8,1.6,1.6,1.6,1.2,1.2,1.2,2.8,2.0,2.0,2.0,1.6,2.0,1.6,2.0,2.0,2.0,2.0,1.6,1.6,0.4,0.4,0.4,2.8,0.4,0.4,0.8,0.8,0.8,0.8,0.4,1.6,2.0,NaN
4,2.4,2.8,2.4,2.4,2.4,2.4,2.8,2.0,1.6,1.2,2.0,0.4,0.4,2.0,2.8,2.8,2.4,0.8,0.8,2.8,2.0,1.6,0.4,0.4,2.4,2.8,1.6,2.0,0.4,1.6,1.6,1.6,2.8,0.4,0.4,0.4,0.4,0.4,2.0,1.6,1.6,0.8,NaN


In [ ]:
from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

In [ ]:
# RBF: Radial-basis function kernel (aka squared-exponential kernel)
#      Length Scale ; A vector with the same number of dimensions as the inputs X (anisotropic variant of the kernel) 
# WhiteKernel: Adds noise to the Kernel
# ConstantKernel: Adding a constant kernel is equivalent to adding a constant.

kernel = ConstantKernel(1.0) + ConstantKernel(1.0) * RBF(length_scale = 41)  + WhiteKernel(5)

# kernel2 = ConstantKernel(1.0) + ConstantKernel(1.0) * 1.0 * Matern(length_scale=1.0, nu=1.5) + WhiteKernel(5)
 # The parameter nu controlling the smoothness of the learned function. 
 # The smaller nu, the less smooth the approximated function is.For nu=inf, the kernel becomes equivalent to the RBF kernel 
 # and for nu=0.5 to the absolute exponential kernel. Important intermediate values are nu=1.5 (once differentiable functions) 
 # and nu=2.5 (twice differentiable functions). Note that values of nu not in [0.5, 1.5, 2.5, inf] incur a considerably higher computational cost 
 # (appr. 10 times higher) since they require to evaluate the modified Bessel function. 
 # Furthermore, in contrast to l, nu is kept fixed to its initial value and not optimized.

#

# n_restarts_optimizer: The number of restarts of the optimizer for finding the kernel's parameters which maximize the log-marginal likelihood.
model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1)
model.fit(X_train_Input, y_train)
y_pred_tr, y_pred_tr_std = model.predict(X_train_Input, return_std=True)
y_pred_te, y_pred_te_std = model.predict(X_test_Input, return_std=True)

In [ ]:
r2_score(y_test, y_pred_te)
# r2_score: R^2 (coefficient of determination) regression score function. 
# An R2 of 0 means your regression is no better than taking the mean value, 
# i.e. you are not using any information from the other variables. 
# A Negative R2 means you are doing worse than the mean value. :))

-0.05507966401230724

In [ ]:
model.score(X_test_Input, y_test)
# = ((y_true - y_pred)** 2).sum()  /  ((y_true - y_true.mean()) ** 2).sum()
# Return the coefficient of determination of the prediction.
# The coefficient of determination  is defined as 
# The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). 
# A constant model that always predicts the expected value of y, disregarding the input features, would get a  score of 0.0.

-0.05507966401230724

In [ ]:
plt.figure()
plt.errorbar(y_test, y_pred_te, yerr=y_pred_te_std, fmt='o')
# r2_score: R^2 (coefficient of determination) regression score function.
plt.title('Gaussian process regression, R2=%.2f' % r2_score(y_test, y_pred_te))
plt.xlabel('Actual')
plt.ylabel('Predicted')

In [ ]:
mean_squared_error(y_train, y_pred_tr)

0.13447651368768565

In [ ]:
mean_squared_error(y_test, y_pred_te)

0.17006510534454336

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test, name="y_test", line_shape='linear'))
fig.add_trace(go.Scatter(y=y_pred_te, name="y_pred_te", line_shape='linear'))
fig.show()

In [ ]:
import statistics
print('NormalDist' in dir(statistics))

False


In [ ]:
idx   = np.argsort(y_test)
y_pred_te = y_pred_te.reshape(38,)
list1a = np.array(y_test)[idx]
list2a = np.array(y_pred_te)[idx]
fig = go.Figure()
fig.add_trace(go.Scatter(y=list1a, name="y_test", line_shape='linear'))
fig.add_trace(go.Scatter(y=list2a, name="y_pred_te", line_shape='linear'))
fig.update_layout(title_text='GAUSSIAN REGRESSION MODEL RESULTS SORTED', title_x=0.5)
fig.show()